In [1]:
%load_ext autoreload
%autoreload 2

import sys
from tqdm.notebook import tqdm
from time import sleep
sys.path.append('../')
from src.commits import *

In [2]:
repo_path = "D:\GitHubs\openstack-neutron"

In [3]:
gr = GitRepository(repo_path)

In [4]:
date_start = datetime(2011, 11, 1)
date_end = datetime(2019, 6, 30)

In [5]:
all_commit_info = {}

In [6]:
num_commits = 0

for commit in RepositoryMining(repo_path, since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
    num_commits += 1
    
print(num_commits)

21547


In [7]:
num_commits_omitted = 0

with tqdm(total = num_commits) as pbar:
    for commit in RepositoryMining(repo_path, since = date_start, to = date_end, only_in_branch = 'master').traverse_commits():
        pbar.set_postfix({'hash' : commit.hash})
        if is_valid_commit(commit):
            all_commit_info[commit.hash] = CommitInfo(commit)
        else:
            num_commits_omitted += 1
        # Progress bar
        pbar.update(1)
        sleep(0.001)

print("{:d} commits ommitted".format(num_commits_omitted))

  0%|          | 0/21547 [00:00<?, ?it/s]

10832 commits ommitted


Get prior and future info

In [8]:
with tqdm(total = len(all_commit_info)) as pbar:
    for commit_hash, commit_info in all_commit_info.items():
        pbar.set_postfix({'hash' : commit_hash})
        update_prior_and_future_info(commit_hash, gr, all_commit_info)
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/10715 [00:00<?, ?it/s]

Save output

In [9]:
all_commit_info_values = []
with tqdm(total = len(all_commit_info)) as pbar:
    for commit_hash, commit_info in all_commit_info.items():
        pbar.set_postfix({'hash' : commit_hash})
        all_commit_info_values.append(commit_info.toDict())
        # Progress bar
        pbar.update(1)
        sleep(0.001)

  0%|          | 0/10715 [00:00<?, ?it/s]

In [10]:
df = pd.DataFrame(data = all_commit_info_values)

In [11]:
df.to_csv("..\data\commits\commits_neutron.csv", index = False)

Clean

In [12]:
df_clean = clean(df)

In [13]:
df_clean.to_csv("..\data\commits\commits_neutron_clean.csv", index = False)